<a href="https://colab.research.google.com/github/dlsanf2000/HIR/blob/main/real_classificationreport_i2b2_on_XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install transformers
import transformers

In [3]:
from tqdm import tqdm, trange

In [4]:
import pickle

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Train.csv') #Set Dir
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Test.csv') #Set Dir

In [6]:
import ast
train_df['words'] = train_df['words'].apply(ast.literal_eval)
train_df['tags'] = train_df['tags'].apply(ast.literal_eval)
test_df['words'] = test_df['words'].apply(ast.literal_eval)
test_df['tags'] = test_df['tags'].apply(ast.literal_eval)

train_words = train_df.words
train_tags = train_df.tags

test_words = test_df.words
test_tags = test_df.tags

In [7]:

New_train_words , New_train_tags =  train_df['words'], train_df['tags']
New_test_words , New_test_tags =  test_df['words'], test_df['tags']

In [8]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [9]:
train_sentences = New_train_words
test_sentences = New_test_words
train_labels =  New_train_tags
test_labels = New_test_tags

In [10]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataframe.csv') #Set Dir

data

,Unnamed: 0,Word,Tag
0,0,Record,O
1,1,date:,O
2,2,2083-10-18,B-DATE
3,3,CARDIOLOGY,O
4,4,WETZEL,B-HOSPITAL
...,...,...,...
815121,815121,Frank,B-DOCTOR
815122,815122,T.,I-DOCTOR
815123,815123,Xuan,I-DOCTOR
815124,815124,",",O


In [11]:
tag_values = list(set(data["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [12]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import XLNetTokenizer, XLNetConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.8.0+cu101'

In [13]:
MAX_LEN = 256
bs = 32

In [14]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [15]:
torch.cuda.get_device_name(0)


'Tesla T4'

In [16]:
!pip install -U transformers
!pip install sentencepiece

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.4.2)


In [17]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=False)


In [18]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [19]:

train_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(train_sentences, train_labels)
]

test_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(test_sentences, test_labels)
]

In [20]:
train_tokenized_texts = [token_label_pair[0] for token_label_pair in train_tokenized_texts_and_labels]
train_labels = [token_label_pair[1] for token_label_pair in train_tokenized_texts_and_labels]

test_tokenize_texts = [token_label_pair[0] for token_label_pair in test_tokenized_texts_and_labels]
test_labels = [token_label_pair[1] for token_label_pair in test_tokenized_texts_and_labels]

In [21]:
test_tokenize_texts[0][:10], test_labels[0][:10]


(['▁Record', '▁date', ':', '▁20', '68', '-', '03', '-', '31', '▁Eu'],
 ['O',
  'O',
  'O',
  'B-DATE',
  'B-DATE',
  'B-DATE',
  'B-DATE',
  'B-DATE',
  'B-DATE',
  'B-HOSPITAL'])

In [22]:

train_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in train_tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in test_tokenize_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [23]:
train_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in train_labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

test_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in test_labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")


In [24]:
train_attention_masks = [[float(i != 0.0) for i in ii] for ii in train_input_ids]
test_attention_masks = [[float(i != 0.0) for i in ii] for ii in test_input_ids]

In [25]:
tr_inputs, val_inputs, tr_tags, val_tags = train_input_ids, test_input_ids, train_tags, test_tags

tr_masks, val_masks = train_attention_masks, test_attention_masks

In [26]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [27]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [28]:

import transformers
from transformers import XLNetForTokenClassification, AdamW

transformers.__version__

'4.4.2'

In [29]:

model = XLNetForTokenClassification.from_pretrained(
    "xlnet-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForTokenClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForTokenClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
model.cuda();


In [31]:
input_size = len(train_data)
num_layers = 3

In [32]:

FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [33]:
from transformers import get_linear_schedule_with_warmup

epochs = 5
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [34]:

!pip install seqeval==0.0.17
from seqeval.metrics import f1_score, accuracy_score,precision_score,recall_score

  Created wheel for seqeval: filename=seqeval-0.0.17-cp37-none-any.whl size=7640 sha256=7ad0121791e58bd38f0e2fbd6d9dcd733f3391ee21bd776bc0586baa61560ce9
  Stored in directory: /root/.cache/pip/wheels/6a/c8/cf/7b9d5d52984c08ce4d27d6f858a682ef74a3738f43f489166a
Successfully built seqeval


In [35]:
## 시간 측정 

import time
start = time.time()


## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print("Validation Recall : {}".format(recall_score(pred_tags, valid_tags)))
    print("Validation Precision : {}".format(precision_score(pred_tags, valid_tags)))
    print()
## 시간 측정 
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Average train loss: 0.22653136894106865
Validation loss: 0.03436529686829696
Validation Accuracy: 0.9908252540656063
Validation F1-Score: 0.9037835718486571
Validation Recall : 0.8880456138999114


Epoch:  20%|██        | 1/5 [03:48<15:13, 228.26s/it]

Validation Precision : 0.9200894104498464

Average train loss: 0.023865863053749006
Validation loss: 0.021155390015337616
Validation Accuracy: 0.9946654570954708
Validation F1-Score: 0.9476674228227635
Validation Recall : 0.9367834022307152


Epoch:  40%|████      | 2/5 [07:37<11:25, 228.59s/it]

...

Validation Precision : 0.963317766335369

Average train loss: 0.009145499705336988
Validation loss: 0.018387708192070324
Validation Accuracy: 0.9957201215507742
Validation F1-Score: 0.957771005659556
Validation Recall : 0.9497272477532279


Epoch:  80%|████████  | 4/5 [15:16<03:48, 228.94s/it]

Validation Precision : 0.965952181375484

Average train loss: 0.0070929581765085455
Validation loss: 0.017334601337400574
Validation Accuracy: 0.9960206591791984
Validation F1-Score: 0.9602064106380868
Validation Recall : 0.9549186799305227


Epoch: 100%|██████████| 5/5 [19:05<00:00, 229.10s/it]

Validation Precision : 0.9655530275815272

time : 1145.5232541561127


In [36]:
# Reset the validation loss for this epoch.
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
predictions , true_labels = [], []
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients,
    # saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have not provided labels.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
    # Move logits and labels to CPU
    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences.
    eval_loss += outputs[0].mean().item()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)

eval_loss = eval_loss / len(valid_dataloader)
validation_loss_values.append(eval_loss)
print("Validation loss: {}".format(eval_loss))
pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                              for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
valid_tags = [tag_values[l_i] for l in true_labels
                              for l_i in l if tag_values[l_i] != "PAD"]
print("Validation Accuracy: {}".format(accuracy_score(valid_tags,pred_tags)))
print("Validation F1-Score: {}".format(f1_score(valid_tags,pred_tags)))
print()

Validation loss: 0.017334601337400574
Validation Accuracy: 0.9960206591791984
Validation F1-Score: 0.9602064106380868



In [37]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

print("Validation Accuracy: {}".format(accuracy_score(valid_tags,pred_tags)))
print("precision_score: {}".format(precision_score(valid_tags,pred_tags)))
print("recall_score: {}".format(recall_score(valid_tags,pred_tags)))
print("Validation F1-Score: {}".format(f1_score(valid_tags,pred_tags)))

Validation Accuracy: 0.9960206591791984
precision_score: 0.9549186799305227
recall_score: 0.9655530275815272
Validation F1-Score: 0.9602064106380868


In [38]:
print(classification_report(valid_tags,pred_tags))

                precision    recall  f1-score   support

           AGE       0.98      0.97      0.98       628
          CITY       0.66      0.82      0.73       390
       COUNTRY       0.81      0.61      0.69       122
          DATE       0.99      0.99      0.99     13251
        DEVICE       0.00      0.00      0.00        29
        DOCTOR       0.96      0.95      0.96      2562
      HOSPITAL       0.81      0.90      0.85      1539
         IDNUM       0.88      0.93      0.90       642
LOCATION-OTHER       0.00      0.00      0.00        15
 MEDICALRECORD       0.98      0.99      0.98      1849
  ORGANIZATION       0.71      0.45      0.55        92
       PATIENT       0.96      0.97      0.97      2135
         PHONE       0.97      0.95      0.96       707
    PROFESSION       0.54      0.51      0.53       136
         STATE       0.88      0.94      0.91       248
        STREET       0.89      0.95      0.92       183
      USERNAME       0.79      0.95      0.86  